In [13]:
from ipywidgets import widgets, interact, interact_manual
import ipywidgets as widgets
from IPython.display import display 
import pandas as pd
import sys
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt

import json
from pandas.io.json import json_normalize
from elasticsearch import Elasticsearch

%matplotlib inline

ModuleNotFoundError: No module named 'elasticsearch'

In [12]:
#Implementing code based on 
#https://medium.com/analytics-vidhya/python-data-projects-data-analysis-ui-reinforced-by-ipywidgets-d680493464b8

## Out 
out = widgets.Output(layout={'border': '1px solid black'})

            
btn_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to Upload',
    icon='check')

btn_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

btn_ch_up= widgets.FileUpload(
    accept='', 
    multiple=False)

ddb_ch_ftype=widgets.Dropdown(options=['.csv','.xlsx'],
                 value='.csv',
                 description= 'Type of file',
                 disable=True)

rbtn_ch_sep=widgets.RadioButtons(
    options=[',',';',' '],
    description='Separator: ',
    disabled=False)

sld_ch_rows=widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

tbtn_ch_describe=widgets.ToggleButtons(
    options=['Preview', 'Info', 'Stats'],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info','tachometer'])

btn_ch_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='danger',
    tooltip='Click to Plot',
    icon='pencil')
ddb_ch_graph_type = widgets.Dropdown(
    options=['Bar Chart', 'Line Chart'],
    value='Bar Chart',
    description='Chart Type:',
    disabled=False)

ddb_ch_file_names = widgets.Dropdown(
    options=[''],
    value='',
    description='File names:',
    disabled=False)

ddb_ch_x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Date:',
    disabled=False)

ddb_ch_y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Height:',
    disabled=False)

cpr_ch_colorsel = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='darkblue',
    disabled=False)

#Define the content of the upload tab
accordion= widgets.Accordion(children=[widgets.VBox([widgets.HBox([ddb_ch_ftype,rbtn_ch_sep,sld_ch_rows]),
                                                     btn_ch_up]),
                                      widgets.VBox([widgets.HBox([ddb_ch_file_names,btn_preview]),out])])

#widgets.link((ddb_ch_ftype,'value'),(btn_ch_up,'accept'))

accordion.set_title(0, 'File selection')
accordion.set_title(1, 'Preview')
accordion_box = widgets.VBox([accordion, 
                             btn_upload])
tab=widgets.Tab()


children=[accordion_box,
         widgets.VBox([tbtn_ch_describe, out]),
         widgets.VBox([
             widgets.HBox([ddb_ch_graph_type, cpr_ch_colorsel]),
             widgets.HBox([ddb_ch_x_axis, ddb_ch_y_axis]),
             btn_ch_plot,
             out])]
tab.children=children
tab.set_title(0, 'Upload Files')
tab.set_title(1, 'Describe')
tab.set_title(2, 'Plotter')

#Data retrieval 
def content_parser():
    if btn_ch_up.value == {}:
        with out:
            print('No File loaded')    
    else:
        typ, content = "", ""
        up_value = btn_ch_up.value
        for i in up_value.keys():
            
            #typ = up_value[i]["metadata"]["type"]        #Original example, not working witht he MIME description, using extension instead 
            typ=i[-4:]
            if typ == ".csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')
                if content_str !='':
                    str_oi=StringIO(content_str)
                    return str_oi

def  df_converter():
    content=content_parser()
    if content is not None:
        #Pending validate xlsx
        print(content, 'Content')
        df=pd.read_csv(content,
                      sep=rbtn_ch_sep.value,
                      index_col=False,
                      skiprows=sld_ch_rows.value)
        return df
    else:
        return None

def plot():
    graph = ddb_ch_graph_type.value
    if graph != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n -----Your {} looks like:----- \n'.format(
                graph))
            if (df is not None):
                df = df.head(5)
                height = df[ddb_ch_y_axis.value]
                bars = df[ddb_ch_x_axis.value]
                y_pos = np.arange(len(height))
                plt.figure(figsize=(10,4))
                if graph == 'Bar Chart':
                    plt.bar(
                        y_pos, 
                        height, 
                        color=color_picker.value)
                    plt.xticks(y_pos, bars)
                elif graph == 'Line Chart':
                    plt.plot(
                        bars,
                        height, 
                        color=color_picker.value,
                        marker='o', 
                        linestyle='solid'
                    )
                    plt.xticks(bars)
                plt.show()
                
def preview():
    df=df_converter()
    with out:
        out.clear_output()
        # if df in locals():
        print('\n-----Now this is how your DF looks like: ----\n')
        if df is not None:
            print(df)
        else:
            print('Config error')
        # else:
        #     print('File not loaded yet')
#@interact
def upload():
    df=df_converter()
    with out:
        out.clear_output()
        print('\n-----Now this is how your DF looks like: ----\n')
        if df is not None:
            print('Estoy aqui\n',df)
            ddb_ch_x_axis.options=df.columns
            ddb_ch_y_axis.options=df.columns
        else:
            print('Configuration is wrong/missing')
            
def preview_clicked(b):
     preview()

def upload_clicked(b):
    upload()

def plotter_clicked(b):
    plot()

btn_ch_plot.on_click(plotter_clicked)
btn_preview.on_click(preview_clicked)
btn_upload.on_click(upload_clicked)



display(tab)

In [5]:
df_converter()

No File loaded


In [76]:
import ipywidgets as widgets
import numpy


output_radio_selected = widgets.Text()
radio1 = widgets.RadioButtons(options=['Option 1'])
radio2 = widgets.RadioButtons(options=['Option 2'])
radio3 = widgets.RadioButtons(options=['Option 3'])
radio4 = widgets.RadioButtons(options=['Option 4'])

radio1.index = None
radio2.index = None
radio3.index = None
radio4.index = None

def radio1_observer(sender):
    #print(sender)
    radio2.unobserve(radio2_observer, names=['value'])
    radio2.index = None

    radio3.unobserve(radio3_observer, names=['value'])
    radio3.index = None

    radio4.unobserve(radio4_observer, names=['value'])
    radio4.index = None

    global selected_option
    output_radio_selected.value = radio1.value
    selected_option = output_radio_selected.value
    print('Selected option set to: ' + selected_option)

    radio2.observe(radio2_observer, names=['value'])
    radio3.observe(radio3_observer, names=['value'])
    radio4.observe(radio4_observer, names=['value'])

def radio2_observer(sender):
    radio1.unobserve(radio1_observer, names=['value'])
    radio1.index = None

    radio3.unobserve(radio3_observer, names=['value'])
    radio3.index = None

    radio4.unobserve(radio4_observer, names=['value'])
    radio4.index = None

    global selected_option2
    output_radio_selected.value = radio2.value
    selected_option2 = output_radio_selected.value
    print('Selected option set to: ' + selected_option2)

    radio1.observe(radio1_observer, names=['value'])
    radio3.observe(radio3_observer, names=['value'])
    radio4.observe(radio4_observer, names=['value'])

def radio3_observer(sender):
    radio1.unobserve(radio1_observer, names=['value'])
    radio1.index = None

    radio2.unobserve(radio2_observer, names=['value'])
    radio2.index = None

    radio4.unobserve(radio4_observer, names=['value'])
    radio4.index = None

    global selected_option3
    output_radio_selected.value = radio3.value
    selected_option3 = output_radio_selected.value
    print('Selected option set to: ' + selected_option3)

    radio1.observe(radio1_observer, names=['value'])
    radio2.observe(radio2_observer, names=['value'])
    radio4.observe(radio4_observer, names=['value'])

def radio4_observer(sender):
    radio1.unobserve(radio1_observer, names=['value'])
    radio1.index = None

    radio2.unobserve(radio2_observer, names=['value'])
    radio2.index = None

    radio3.unobserve(radio3_observer, names=['value'])
    radio3.index = None

    global selected_option4
    output_radio_selected.value = radio4.value
    selected_option4 = output_radio_selected.value
    print('Selected option set to: ' + selected_option4)

    radio1.observe(radio1_observer, names=['value'])
    radio2.observe(radio2_observer, names=['value'])
    radio3.observe(radio3_observer, names=['value'])

radio1.observe(radio1_observer, names=['value'])
radio2.observe(radio2_observer, names=['value'])
radio3.observe(radio3_observer, names=['value'])
radio4.observe(radio4_observer, names=['value'])

widgets.HBox([radio1,radio2,radio3,radio4])